In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from transition_util import *
import multiprocessing as mp
import pickle
import math
import os

In [2]:
flight_data = pd.read_csv("../../samples_with_jfk_landing_flag.csv")
flight_data = flight_data.sort_values(by = ['id', 'ts']).reset_index(drop = True)
# flight_data['jfk_landing_flag'] = (flight_data['jfk_landing_flag'] & (flight_data['destination'] == 'JFK'))
flight_data['lon'] = flight_data['lon'].clip(-78, -68)
flight_data['lat'] = flight_data['lat'].clip(35, 45)
flight_data['altitude'] = flight_data['altitude'].clip(0, 50000)
conversion_x = 9.26/((flight_data['lat'] * math.pi / 180).apply(math.cos) * 111.699)
conversion_y = 9.26/111.699
conversion_z = 500
flight_data['x'] = ((flight_data['lon'] + 78)/conversion_x).apply(np.ceil).apply(int)
flight_data['y'] = ((flight_data['lat'] - 35)/conversion_y).apply(np.ceil).apply(int)
flight_data['z'] = ((flight_data['altitude']) / conversion_z).apply(np.ceil).apply(int)

MemoryError: 

In [ ]:
print(flight_data['x'].max())
print(flight_data['x'].min())
print(flight_data['y'].max())
print(flight_data['y'].min())
print(flight_data['z'].max())
print(flight_data['z'].min())

In [ ]:
start_pos_df = flight_data.groupby(['id']).first().apply(lambda x: (x['x'], x['y'], x['z']), axis = 1).reset_index(drop = False)
start_pos_df.columns = ['id', 'start_pos']
flight_data = flight_data.merge(start_pos_df)
flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)
flight_data.head()

In [ ]:
flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)

In [ ]:
if not(os.path.isfile("transitions1_start_xyz.pkl")):
    transitions1_start_xyz = {}
    delta_start_ts1 = {}
    condition = True
    previous_id = ''
    flight_data1 = flight_data
    flight_data1['last_ts'] = flight_data1[['id', 'ts']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_x'] = flight_data1[['id', 'x']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_y'] = flight_data1[['id', 'y']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_z'] = flight_data1[['id', 'z']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_lon'] = flight_data1[['id', 'lon']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_lat'] = flight_data1[['id', 'lat']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_climb_rate'] = flight_data1[['id', 'climb_rate']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_altitude'] = flight_data1[['id', 'altitude']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1 = flight_data1[~flight_data1['last_ts'].apply(np.isnan)]
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data1 = flight_data1[(flight_data1['last_x'] != flight_data1['x']) |
                              (flight_data1['last_y'] != flight_data1['y']) |
                              (flight_data1['last_z'] != flight_data1['z'])]
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data1['last_x'] = flight_data1['last_x'].apply(int)
    flight_data1['last_y'] = flight_data1['last_y'].apply(int)
    flight_data1['last_z'] = flight_data1['last_z'].apply(int)
    
    unit_time = 5
    start_df = flight_data1[['id', 'last_ts', 'last_lon', 'last_lat', 'last_altitude']].groupby(['id']).first().reset_index(drop = False)
    start_df.columns = ['id', 'start_ts', 'start_lon', 'start_lat', 'start_altitude']
    flight_data1 = flight_data1.merge(start_df)
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data1['t'] = ((flight_data1['ts'] - flight_data1['start_ts'])/unit_time).apply(np.ceil).apply(int)
    flight_data1['last_t'] = ((flight_data1['last_ts'] - flight_data1['start_ts'])/unit_time).apply(np.ceil).apply(int)
    
    # Retaining only aircraft IDs with destination == "JFK" in all rows
    df1 = flight_data[flight_data['jfk_landing_flag']]
    df1 = df1.sort_values(['id', 'ts']).reset_index(drop = True)
    df1 = df1.groupby(['id']).apply(lambda x: (x['destination'] == "JFK").mean()).reset_index(drop = False)
    df1 = df1[['id']][df1[0] == 1.0]
    landing_ac_data = df1.merge(flight_data1)
    landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
    
    # Retaining only IDs that have successfully landed at JFK (small range)
    last_df = landing_ac_data.groupby(['id']).last().reset_index(drop = False)
    last_df['coords'] = last_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
    last_df = last_df[last_df['coords'].apply(lambda x: x[2] <= 3)]
    print(last_df['coords'].value_counts())
    landing_ac_data = landing_ac_data.merge(last_df[['id']])
    landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
    landed_xyz = set(last_df['coords'])
    print(landed_xyz)
    
    for i in range(landing_ac_data.shape[0]):
        id1 = landing_ac_data['id'].iloc[i]
        start_pos = landing_ac_data['start_pos'].iloc[i]
        last_x = landing_ac_data['last_x'].iloc[i]
        last_y = landing_ac_data['last_y'].iloc[i]
        last_z = landing_ac_data['last_z'].iloc[i]
        last_ts = landing_ac_data['last_ts'].iloc[i]
        x = landing_ac_data['x'].iloc[i]
        y = landing_ac_data['y'].iloc[i]
        z = landing_ac_data['z'].iloc[i]
        ts = landing_ac_data['ts'].iloc[i]
        condition = (previous_id != id1)
        if (not((last_x, last_y, last_z) in landed_xyz) and condition):
            last_row = True
            try:
                transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] += 1
                delta_start_ts1[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] += [ts - last_ts]
            except:
                try:
                    transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] = 1
                    delta_start_ts1[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] = [ts - last_ts]
                except:
                    transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): 1}
                    delta_start_ts1[(start_pos, (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): [ts - last_ts]}
        elif last_row:
            previous_id = id1
            last_row = False
            try:
                transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] += 1
                delta_start_ts1[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] += [ts - last_ts]
            except:
                try:
                    transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] = 1
                    delta_start_ts1[(start_pos, (last_x, last_y, last_z))][(start_pos, (x, y, z))] = [ts - last_ts]
                except:
                    transitions1_start_xyz[(start_pos, (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): 1}
                    delta_start_ts1[(start_pos, (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): [ts - last_ts]}
        else:
            print(previous_id)
    
    pickle.dump(delta_start_ts1, open("delta_ts1_start.pkl", "wb"))
    pickle.dump(transitions1_start_xyz, open("transitions1_start_xyz.pkl", "wb"))
    pickle.dump(landed_xyz, open("landed_start_xyz.pkl", "wb"))
else:
    transitions1_start_xyz = pickle.load(open("transitions1_start_xyz.pkl", "rb"))
    delta_start_ts1 = pickle.load(open("delta_ts1_start.pkl", "rb"))
    landed_xyz = pickle.load(open("landed_start_xyz.pkl", "rb"))

In [ ]:
if not(os.path.isfile("transitions2_start_xyz.pkl")):
    flight_data2 = flight_data1
    flight_data2['last_last_ts'] = flight_data2[['id', 'last_ts']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_x'] = flight_data2[['id', 'last_x']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_y'] = flight_data2[['id', 'last_y']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_z'] = flight_data2[['id', 'last_z']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_lon'] = flight_data2[['id', 'last_lon']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_lat'] = flight_data2[['id', 'last_lat']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_altitude'] = flight_data2[['id', 'last_altitude']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2['last_last_climb_rate'] = flight_data2[['id', 'last_climb_rate']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data2 = flight_data2[~flight_data2['last_last_ts'].apply(np.isnan)]
    flight_data2 = flight_data2.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data2['last_last_x'] = flight_data2['last_last_x'].apply(int)
    flight_data2['last_last_y'] = flight_data2['last_last_y'].apply(int)
    flight_data2['last_last_z'] = flight_data2['last_last_z'].apply(int)
    flight_data2['last_last_t'] = ((flight_data2['last_last_ts'] - flight_data2['start_ts'])/unit_time).apply(np.ceil).apply(int)

    landing_ac_data2 = df1.merge(flight_data2[flight_data2['jfk_landing_flag']])
    landing_ac_data2 = landing_ac_data2.sort_values(['id', 'ts']).reset_index(drop = True)
    landing_ac_data2.shape
    transitions2_start_xyz = {}
    delta_start_ts2 = {}
    previous_id = ''
    for i in range(landing_ac_data2.shape[0]):
        last_last_x = landing_ac_data2['last_last_x'].iloc[i]
        last_last_y = landing_ac_data2['last_last_y'].iloc[i]
        last_last_z = landing_ac_data2['last_last_z'].iloc[i]
        last_last_ts = landing_ac_data2['last_last_ts'].iloc[i]
        last_x = landing_ac_data2['last_x'].iloc[i]
        last_y = landing_ac_data2['last_y'].iloc[i]
        last_z = landing_ac_data2['last_z'].iloc[i]
        last_ts = landing_ac_data2['last_ts'].iloc[i]
        id1 = landing_ac_data2['id'].iloc[i]
        start_pos = landing_ac_data2['start_pos'].iloc[i]
        x = landing_ac_data2['x'].iloc[i]
        y = landing_ac_data2['y'].iloc[i]
        z = landing_ac_data2['z'].iloc[i]
        ts = landing_ac_data2['ts'].iloc[i]
        condition = (previous_id != id1)
        if not((x, y, z) in landed_xyz) and condition:
            last_row = True
            try:
                transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] += 1
                delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] += [ts - last_ts]
            except:
                try:
                    transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] = 1
                    delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] = [ts - last_ts]
                except:
                    transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): 1}
                    delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): [ts - last_ts]}
        elif last_row:
            previous_id = id1
            last_row = False
            try:
                transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] += 1
                delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] += [ts - last_ts]
            except:
                try:
                    transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] = 1
                    delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))][(start_pos, (x, y, z))] = [ts - last_ts]
                except:
                    transitions2_start_xyz[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): 1}
                    delta_start_ts2[(start_pos, (last_last_x, last_last_y, last_last_z), (last_x, last_y, last_z))] = {(start_pos, (x, y, z)): [ts - last_ts]}
        else:
            print(previous_id)
    
    pickle.dump(transitions2_start_xyz, open("transitions2_start_xyz.pkl", "wb"))
    pickle.dump(delta_start_ts2, open("delta_ts2_start.pkl", "wb"))
else:
    transitions2_start_xyz = pickle.load(open("transitions2_start_xyz.pkl", "rb"))
    delta_start_ts2 = pickle.load(open("delta_ts2_start.pkl", "rb"))

In [8]:
lis1 = [transitions2_start_xyz[key].values() for key in transitions2_start_xyz.keys()]
lis2 = unlist(lis1)
max(lis2)

81

In [9]:
print(flight_data.shape)
start_df = flight_data[['id', 'ts']].groupby(['id']).first().reset_index(drop = False)
end_df = flight_data[['id', 'ts']].groupby(['id']).last().reset_index(drop = False)
start_df.columns = ['id', 'id_start_ts']
end_df.columns = ['id', 'id_end_ts']
flight_data = flight_data.merge(start_df)
flight_data = flight_data.merge(end_df)
flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)
print(flight_data.shape)

(44023466, 25)
(44023466, 27)


In [10]:
df1 = flight_data[flight_data['jfk_landing_flag']]
df1 = df1.sort_values(['id', 'ts']).reset_index(drop = True)
df1 = df1.groupby(['id']).apply(lambda x: (x['destination'] == "JFK").mean()).reset_index(drop = False)
df1 = df1[['id']][df1[0] == 1.0]
landing_ac_data = df1.merge(flight_data)
landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)

# Retaining only IDs that have successfully landed at JFK (small range)
last_df = landing_ac_data.groupby(['id']).last().reset_index(drop = False)
last_df['coords'] = last_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
last_df = last_df[last_df['coords'].apply(lambda x: x[2] <= 3)]
print(last_df['coords'].value_counts())
landing_ac_data = landing_ac_data.merge(last_df[['id']])
landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)

(39, 68, 0)    3881
(39, 69, 0)    2950
(40, 68, 0)       2
(40, 69, 0)       1
Name: coords, dtype: int64


In [11]:
# Simulating landing of 1 aircraft
# uniq_id = landing_ac_data['id'].unique()
# i = random.randint(a = 0, b = len(uniq_id) - 1)
# chosen_id = uniq_id[i]
chosen_id = "A033AE_47"
df = flight_data[flight_data['id'] == chosen_id].sort_values(['id', 'ts']).reset_index(drop = True).iloc[0]
start_point = (df['x'], df['y'], df['z'])
df

id                    A033AE_47
aircraft                 A033AE
lat                      36.006
lon                         -78
ts                   1560657958
altitude                  37000
climb_rate                  192
azimuth                      58
ground_speed                458
jfk_landing_flag           True
source                      LAX
destination                 JFK
ground_flag                   0
x                             0
y                            13
z                            74
start_pos           (0, 13, 74)
last_ts                     NaN
last_x                      NaN
last_y                      NaN
last_z                      NaN
last_lon                    NaN
last_lat                    NaN
last_climb_rate             NaN
last_altitude               NaN
id_start_ts          1560657958
id_end_ts            1560661589
Name: 0, dtype: object

In [12]:
total_times = []
paths = []
max_landings = 10
landings = 0

def simulate_landing(df):
    landed = False
    total_time = 0
    path = []
    while not landed:
        path.append((df['x'], df['y'], df['z']))
        ts1 = df['ts']
#         print(ts1)
        # We will not be able to know the complete duration of chosen aircraft before it actually lands
        temp_flight_data = flight_data[(flight_data['id_start_ts'] <= ts1) &
                                       (flight_data['id_end_ts'] >= ts1) &
                                       (flight_data['id'] != chosen_id)]
        last_known_df = temp_flight_data[temp_flight_data['ts'] <= ts1].reset_index(drop = True).groupby(['id']).last().reset_index(drop = False)
        last_known_df = last_known_df[(last_known_df['altitude'] > 0) | (last_known_df['climb_rate'] != 0)].reset_index(drop = True)
        current_df = last_known_df.apply(lambda x: get_current_pos(x, ts1), axis = 1)
#         current_df = current_df[]
        current_pos = current_df.apply(get_xyz)
        current_pos = current_pos[current_pos.apply(isinrange)]
        if total_time == 0:
            actions = transitions1_start_xyz[(start_point, (df['x'], df['y'], df['z']))]
            temp_delta_ts = delta_start_ts1[(start_point, (df['x'], df['y'], df['z']))]
        else:
            actions = transitions2_start_xyz[(start_point, previous_state, (df['x'], df['y'], df['z']))]
            temp_delta_ts = delta_start_ts2[(start_point, previous_state, (df['x'], df['y'], df['z']))]
        next_xyzs = [repeat(key, actions[key]) for key in actions.keys()]
        next_xyzs = unlist(next_xyzs)
        conflict = True
        tries = 0
        # This should probably change to greedy selection
        while conflict:
            j = random.randint(a = 0, b = len(next_xyzs) - 1)
            next_xyz = next_xyzs[j]
            all_delta_ts = temp_delta_ts[next_xyz]
            # This should probably change to greedy selection
            j = random.randint(a = 0, b = len(all_delta_ts) - 1)
            temp_delta_ts = all_delta_ts[j]
            df['ts'] = df['ts'] + temp_delta_ts
            previous_state = (df['x'], df['y'], df['z'])
            total_time += temp_delta_ts
            conflict = any(current_pos.apply(lambda x: check_conflict(x, next_xyz[len(next_xyz) - 1])))
            tries += 1
            if tries > 10:
                print("Conflict unlikely to be avoided at " + str((df['x'], df['y'], df['z'])))
        
        df['x'] = next_xyz[len(next_xyz) - 1][0]
        df['y'] = next_xyz[len(next_xyz) - 1][1]
        df['z'] = next_xyz[len(next_xyz) - 1][2]
        
        landed = (df['x'], df['y'], df['z']) in landed_xyz
    return [path, total_time]

def simulate_landing_i(i):
    return simulate_landing(df)

In [ ]:
# cores = max_landings
# p = mp.Pool(processes = cores)
# split_dfs = np.array_split([i for i in range(cores)], cores)
# pool_results = p.map(simulate_landing_i, split_dfs)
# p.close()
# p.join()

In [ ]:
# p.close()
# p.join()

In [18]:
y_max = flight_data['y'].max()
x_max = flight_data['x'].max()

In [33]:
landed = False
total_time = 0
path = []
while not landed:
    path.append((df['x'], df['y'], df['z']))
    ts1 = df['ts']
#         print(ts1)
    # We will not be able to know the complete duration of chosen aircraft before it actually lands
    temp_flight_data = flight_data[(flight_data['id_start_ts'] <= ts1) &
                                   (flight_data['id_end_ts'] >= ts1) &
                                   (flight_data['id'] != chosen_id)]
    last_known_df = temp_flight_data[temp_flight_data['ts'] <= ts1].reset_index(drop = True).groupby(['id']).last().reset_index(drop = False)
    last_known_df = last_known_df[(last_known_df['altitude'] > 0) | (last_known_df['climb_rate'] != 0)].reset_index(drop = True)
    current_df = last_known_df.apply(lambda x: get_current_pos(x, ts1), axis = 1)
#         current_df = current_df[]
    current_pos = current_df.apply(get_xyz)
    current_pos = current_pos[current_pos.apply(lambda x: isinrange(x, lat_len = y_max, lon_len = x_max, alt_len = 100))]
    conflict = True
    tries = 0
    # This should probably change to greedy selection
    while conflict:
        if total_time == 0:
            actions = transitions1_start_xyz[(start_point, (df['x'], df['y'], df['z']))]
            temp_delta_ts = delta_start_ts1[(start_point, (df['x'], df['y'], df['z']))]
        else:
            actions = transitions2_start_xyz[(start_point, previous_state, (df['x'], df['y'], df['z']))]
            temp_delta_ts = delta_start_ts2[(start_point, previous_state, (df['x'], df['y'], df['z']))]
#         next_xyzs = [repeat(key, actions[key]) for key in actions.keys()]
#         next_xyzs = unlist(next_xyzs)
#         j = random.randint(a = 0, b = len(next_xyzs) - 1)
#         next_xyz = next_xyzs[j]
#         all_delta_ts = temp_delta_ts[next_xyz]
        # This should probably change to greedy selection
#         j = random.randint(a = 0, b = len(all_delta_ts) - 1)
#         temp_delta_ts = all_delta_ts[j]
        next_xyz = sorted(actions.items(), key = lambda kv:(kv[1], kv[0]))
        future_df = last_known_df.apply(lambda x: get_current_pos(x, ts1 + temp_delta_ts), axis = 1)
        future_pos = future_df.apply(get_xyz)
        future_pos = future_pos[future_pos.apply(lambda x: isinrange(x, lat_len = y_max, lon_len = x_max, alt_len = 100))]
        conflict = any(future_pos.apply(lambda x: check_conflict(x, next_xyz[len(next_xyz) - 1])))
        tries += 1
        if tries > 10:
            print("Conflict unlikely to be avoided at " + str((df['x'], df['y'], df['z'])))
            conflict = False
            landed = True
    
    df['ts'] = df['ts'] + temp_delta_ts
    previous_state = (df['x'], df['y'], df['z'])
    total_time += temp_delta_ts
    df['x'] = next_xyz[len(next_xyz) - 1][0]
    df['y'] = next_xyz[len(next_xyz) - 1][1]
    df['z'] = next_xyz[len(next_xyz) - 1][2]
    print(next_xyz[len(next_xyz) - 1])

    landed = (df['x'], df['y'], df['z']) in landed_xyz

TypeError: ("unsupported operand type(s) for +: 'float' and 'dict'", 'occurred at index 0')

In [34]:
next_xyz

[(((0, 13, 74), (42, 71, 8)), 1)]

In [20]:
all_delta_ts

[16.0]

In [32]:
future_df = last_known_df.apply(lambda x: get_current_pos(x, ts1 + temp_delta_ts), axis = 1)
future_pos = future_df.apply(get_xyz)
subset = future_pos.apply(lambda x: isinrange(x, lat_len = y_max, lon_len = x_max, alt_len = 100))
last_known_df = last_known_df[subset]
future_pos = future_pos[subset]
future_df = future_df[subset]
conflict = future_pos.apply(lambda x: check_conflict(x, next_xyz[len(next_xyz) - 1]))
# print(conflict.shape)
# print(last_known_df[conflict])
print(last_known_df[conflict])

          id aircraft      lat      lon          ts  altitude  climb_rate  \
17  780A15_0   780A15  40.8024 -73.4045  1560659864      4000           0   

    azimuth  ground_speed  jfk_landing_flag  ...       last_ts last_x  last_y  \
17      302           210              True  ...  1.560660e+09   43.0    70.0   

    last_z  last_lon  last_lat last_climb_rate  last_altitude  id_start_ts  \
17     8.0   -73.397   40.7988           -64.0         4000.0   1560657251   

     id_end_ts  
17  1560660439  

[1 rows x 27 columns]


In [ ]:
next_xyzs = [repeat(key, actions[key]) for key in actions.keys()]

In [ ]:
next_xyzs

In [ ]:
future_pos[future_pos.apply(lambda x: check_conflict(x, next_xyz[len(next_xyz) - 1]))]

In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# times = []
# for i in range(len(pool_results)):
#     res = pool_results[i]
#     path = res[0]
#     time = res[1]
#     times.append(time)
#     mat = np.zeros((101, 101, 101))
#     for coord in path:
#         mat[coord[0], coord[1], coord[2]] += 1

#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     pos = np.where(mat > 0)
#     ax.scatter(pos[0], pos[1], pos[2], c='black')
#     plt.show()

In [ ]:
# times